In [23]:



import pandas as pd
import os
import json

dir = '../../gym/raw/llm_json/gpt4o'

queries = {
    1: '(Graph structured) AND (visual Question Answering)',
    2: '"Autonomous Surface Vehicles" AND "Federated Learning"',
    3: 'RAG AND (Long-context LLM) OR (long context language model)',
    4: 'Applications of visual question answering systems',
    5: 'Fairness AND socials',
    6: '(Regulatory Capture AND IA OR ML) OR (Regulatory Capture OR Machine Learning)',
    7: '(Segmentation OR SAM) AND failure',
    8: '(Graph Convolutional Network) OR (GCN) AND ',
    9: '((Question Answering) AND (visual)) OR (VQA)',
    10: 'FL AND decentralised',
}

In [24]:


results = []
for file in os.listdir(dir):
    with open(f'{dir}/{file}', 'r') as f:
        response = f.read()
        
        # remove all that is not json
        response = response.split('{')[1].split('}')[0]
        response = '{' + response + '}'
                
        try:
            response = json.loads(response)
        except json.JSONDecodeError as e:
            print(e)
            print(f'Error in file {file}')
            continue

        score = response.get('score', None)
        relevant = response.get('relevant', None)
        explanation = response.get('explanation', None)
        keywords = response.get('keywords', None)
        
       
        index, query = file.split('_')
        query = query.split('.')[0]
  

        results.append({
            'index': index,
            'query_id': query,
            'score': score,
            'relevant': relevant,
            'explanation': explanation,
            'keywords': keywords
        })

llm_res = pd.DataFrame(results)
llm_res

,index,query_id,score,relevant,explanation,keywords
0,0,1,0.0,False,No match found in table content.,"[Graph structured, visual Question Answering]"
1,0,2,0.0,False,No keywords matching the query found in the ta...,[]
2,0,3,0.0,False,"No mention of RAG, Long-context LLM, or long c...",[]
3,0,4,0.0,False,Table appears to discuss test accuracy of mach...,[]
4,0,5,0.0,False,No match found for fairness and socials,[]
...,...,...,...,...,...,...
1742,175,6,0.0,False,"No match found for Regulatory Capture, IA, ML,...",[]
1743,175,7,0.0,False,No keywords from the query present in the tabl...,[]
1744,175,8,0.0,False,No mention of Graph Convolutional Network or n...,[]
1745,175,9,0.0,False,Table is unrelated to Question Answering or Vi...,[]


In [25]:


data = pd.read_csv('../../gym/raw/data.csv')

llm_res['index'] = llm_res['index'].astype(int)
data['index'] = data['index'].astype(int)

llm_res = llm_res.merge(data, on='index', how='left')

llm_res['id'] = llm_res['file_id'].astype(str) + '_' + llm_res['table_id'].astype(str)
llm_res.to_csv('../../gym/results/gpt4o/llm_response.csv', index=False)


